In [54]:
import os
import torch
import numpy as np
import cv2

In [7]:
import re

In [2]:
p = '/scratch/users/jpierre'
os.listdir(p)

['mew_0.01',
 'mew_noise',
 'mew_noise_0.01',
 'mew',
 'mew_0.01_bis',
 'mew_0.001_noisy',
 'mew_0.001_normal',
 'mew_0.01_noisy',
 'mew_0.01_normal']

In [4]:
dataset = 'mew_0.001_normal'
torch_dataset = f'/scratch/users/jpierre/{dataset}/training/torch_file'
np_dataset = f'/scratch/users/jpierre/{dataset}/training/np_file'
df_file = f'/scratch/users/jpierre/{dataset}'

In [5]:
data = os.listdir(torch_dataset)

In [6]:
print(data[0])

sim_0_step_0.pt


In [21]:
def collect_steps(path, simulation_number):
    pattern = re.compile(f"sim_{simulation_number}_step_(\d+)\.pt")

    steps = []; res = []

    # Iterate over all files in the directory
    for filename in os.listdir(path):
        # Check if the file matches the pattern
        match = pattern.match(filename)
        if match:
            step = int(match.group(1))
            steps.append(step)
    
    steps.sort()
    
    for i in range(len(steps)):
        res.append(torch.load(os.path.join(path, f"sim_{simulation_number}_step_{steps[i]}.pt")))
    
    return res

In [22]:
r = collect_steps(torch_dataset, 0)

/home/jpierre/anaconda3/envs/myenvPy/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
print(r[0])

Data(x=[300, 10], edge_index=[2, 344], edge_attr=[344, 5], y=[300, 10, 2])


In [79]:
def display_out_grpah(data_graph, video_params, bounds=None):
    """
    Creates an MP4 video from a PyTorch tensor representing graph nodes and edges using cv2,
    based on specified video parameters and position bounds.

    Parameters:
    - data_x: List of input features of the graph
    - edge_indices: List of tensors for the different indices
    - video_params: An instance of VideoParameters class containing video settings.
    - bounds: Tuple of ((min_x, max_x), (min_y, max_y)) specifying the bounds for the positions.
              If None, uses the minimum and maximum values from the data.
    """


    ## get back the array with teh positions

    mat_pos = []
    mat_indices = []

    for i in range(len(data_graph)):
        mat_pos.append(data_graph[i].x[:, :2])

        mat_indices.append(data_graph[i].edge_index)    


    mat_pos = np.stack(mat_pos, axis = 0)  
    print(len(mat_indices))
    print(mat_pos.shape)
        
    if bounds:
        min_x, max_x = bounds[0]
        min_y, max_y = bounds[1]
    else:
        min_x, max_x = mat_pos[:, :, 0].min(), mat_pos[:, :, 0].max()
        min_y, max_y = mat_pos[:, :, 1].min(), mat_pos[:, :, 1].max()

    # Normalize coordinates to fit within the video frame size
    mat_pos[:, :, 0] = (mat_pos[:, :, 0] - min_x) / (max_x - min_x) * (video_params.size[0] - 1)
    mat_pos[:, :, 1] = (mat_pos[:, :, 1] - min_y) / (max_y - min_y) * (video_params.size[1] - 1)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Be sure to use lowercase
    out = cv2.VideoWriter(video_params.path, fourcc, video_params.fps, video_params.size)
    
    mat_pos = mat_pos.astype(int)

    for i in range(mat_pos.shape[0]):
        frame = np.zeros((video_params.size[1], video_params.size[0], 3), dtype=np.uint8)

        # Draw nodes
        for x, y in mat_pos[i]:
            cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)

        # Draw edges
        for start_node, end_node in mat_indices[i].t().numpy():
            pt1 = tuple(mat_pos[i, start_node, :])
            pt2 = tuple(mat_pos[i, end_node, :])
            cv2.line(frame, pt1, pt2, (0, 0, 255), 2)


        out.write(frame)

    out.release()

In [80]:
CFG_VIDEO = {'radius': 5, 'color': (0, 255, 0)}


class videoParameters():
    def __init__(self, path:str, params = CFG_VIDEO):
        """ 
        Args:
        -----
            - `path`: path of the video
        """
        self.fps = 10
        self.path = path
        self.size = (600, 600)
        self.params = params

In [81]:
p = videoParameters(path = os.path.join(os.getcwd(), 'test.mp4'))

In [82]:
display_out_grpah(r, p)



989
(989, 300, 2)
